In [1]:
import sys
import os

model_upgrading_path = os.path.join("..","src")
sys.path.append(model_upgrading_path)

In [2]:
# moduel
import my_package.time_series as time
from my_package.data.select_ship_dataset import get_ship_dataframe_from_database
from my_package.data.load_database import load_database
from my_package.data.select_dataset_all import get_all_dataframe_from_database
from my_package.data.delete_from_database import delete_from_database

# basic 
import pandas as pd
import numpy as np
import pickle
from datetime import datetime

# visualize
import matplotlib.pyplot as plt  

# learning
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

ModuleNotFoundError: No module named 'my_package.data.select_dataset_all'

클래스

1. input: 선박 이름, output: 현재 op를 기반으로 다음 op의 전극 효율 평균 예측 )
2. ConfiguringTimeData + TimeSeriesModel

In [3]:
class ConfiguringTimeData:
   # 인풋 데이터 구축
    def __init__(self, ship_name):
        # 선박 이름을 이용한 선박 아이디 추출 
        ship_id = ship_id = time.find_ship(ship_name)[0]
        
        # 데이터 로드 (과거 데이터)
        self.ship_data = get_ship_dataframe_from_database('tc_ai_current_system_health', ship_id)
        
        # 해당 선박 데이터 생성
        self.create_electrode_data()
        
        # 해당 선박 정제
        self.preprocess_data()
        
        # 해당 선박 전극 효율 이동평균 생성
        self.moving_average_df = time.generate_moving_average(self.electrod_df, 250)
        
        # 해당 선박 시간 그룹화
        self.group_time_series()
        
        # 해당 테스트 데이터 생성
        #self.test()
        
        # 부족한 시간 계산 및 학습 데이터 생성
        #self.calculate_the_time()
        
    def create_electrode_data(self):
        """ 데이터 로드 및 전처리
        """
        # 1 .전처리
        data  = self.ship_data[self.ship_data['ELECTRODE_EFFICIENCY'].notna()]
        data = data[(data['ELECTRODE_EFFICIENCY']>-100)]
        data = data.drop_duplicates()
        
        # 2. 결측치 제거
        self.data = data.dropna()

    
    def group_time_series(self):
        """
         시간 기준 데이터 그룹화( 효율 평균 )
        """
    
        # 1. 시간을 기준으로 전극 효율 계산
        self.historic_data = self.moving_average_df[['DATA_TIME','Moving_Average']].groupby('DATA_TIME').mean().reset_index()

        # 2. datetime 형식으로 변환
        self.historic_data['DATA_TIME'] = pd.to_datetime(self.historic_data['DATA_TIME'])
    
    
    def preprocess_data(self):
        """ 데이터 정제 및 처리
        """
        # 1. 순서 정렬
        electrod_df = self.data.sort_values(by=['DATA_TIME'])
        
        # 2.시간을 년-월-일-시-분으로 추출
        electrod_df['DATA_TIME'] = electrod_df['DATA_TIME'].dt.strftime('%Y-%m-%d %H:%M')

        # 3. 인덱스 재 설정
        electrod_df = electrod_df.reset_index(drop=True)

        # 4. 중복 값 제거
        self.electrod_df = electrod_df.drop_duplicates()
    
    
    def calculate_the_time(self):
        """
            부족한 시간 추출
        """
        
        # 1. 학습을 위한 요구 시간 30h
        required_time = 30
        
        # 2. 현재 시간을 계산
        total_minutes = len(self.test_curr_data)
        total_hours = np.round(total_minutes / 60,2)
        remaining_hours = required_time - total_hours
        remaining_minutes = int(remaining_hours * 60)  # 1055분
        
        # 3. 현재 시간에서 필요한 시간을 과거 데이터에서 추출
        recent_data = self.historic_test_data.iloc[-remaining_minutes:].copy()
        
        # 4. 기존 데이터와 복사된 데이터를 결합
        self.extended_df = pd.concat([recent_data, self.test_curr_data], ignore_index=True)
    
    def test(self):
        """ 테스트 
        """
        # 데이터 생성
        
        # 1. 전체 데이터 길이
        data_len = len(self.historic_data)

        # 2. 과거 데이터 길이 및 데이터
        
        historic_data_len = int((data_len) * 0.99)
        self.historic_test_data = self.historic_data[:historic_data_len]

        # 3. 현재 데이터 
        self.test_curr_data = self.historic_data[historic_data_len:]
        
    
    def group_3h_time_series(self):
        """ 분 → 3h으로 리 셈플링
        """
        # . 현재 시간에서 필요한 시간을 과거 데이터에서 추출
        remaining_minutes = int(30 * 60)  # 1055분
        recent_data = self.historic_data.iloc[-remaining_minutes:].copy()
        
        # 1. 데이터 길이 및 그룹 사이즈
        data_length  = len(recent_data)
        group_size = 180

        # 2. 그룹 번호를 1부터 시작하도록 수정 (0부터 시작하는 그룹에 1을 더함)
        groups = (np.arange(data_length) // group_size) + 1
        
        # 3. 그룹 변수 생성
        recent_data['group'] = groups

        # 4.그룹별로 원하는 집계 수행 (예: 평균)
        grouped = recent_data.groupby('group').agg({'DATA_TIME': 'first', 'Moving_Average': 'mean'})
        
        grouped.reset_index(drop=True, inplace=True)
        
        return grouped

In [4]:
class TimeSeriesModel:
    def __init__(self, data):
        # 파일 경로를 지정하여 모델을 로드
        
        # 현재파 파일의 경로를 기준으로 model 폴더 내 모델 및 스케일러 경로 생성
        
        current_dir = os.getcwd()
        time_model_relative_path = os.path.join(current_dir,".." , 'src','my_package',"model","time_model_250_3h_10.pkl")
        scaler_model_relative_path = os.path.join(current_dir,".." , 'src','my_package',"scaler")
        
        # 상대 경로를 절대 경로로 변환
        time_model_path = os.path.abspath(time_model_relative_path)
        scaler_model_path = os.path.abspath(scaler_model_relative_path)
        
        self.model = time.load_model_from_pickle(time_model_path)
        self.scaler = time.load_model_from_pickle(scaler_model_path)
        self.fit_into_data_format(data)
        
    def fit_into_data_format(self, data):
         # 데이터 스케일링
        scaled_data = self.scaler.fit_transform(data[['Moving_Average']])
        
        # 데이터 형태 조정 (LSTM input 형식)
        self.X = scaled_data.reshape((scaled_data.shape[0], scaled_data.shape[1], 1))
        
        print(self.X)
        
    def predict(self):
        
        predictions = self.model.predict(self.X[0])
        predictions = self.scaler.inverse_transform(predictions.reshape(-1, 1))
        #y_test_rescaled = self.scaler.inverse_transform(y_test.reshape(-1, 1))
        
        return predictions.item()

함수

In [5]:
def create_dataset(ship_id, pred):
    data = get_all_dataframe_from_database('tc_ai_ecu_cleaning_cycle_model')
    
    # SHIP_ID가 존재하면 업데이트, 존재하지 않으면 새로운 행 추가
    if ship_id in data['SHIP_ID'].values:
        data.loc[data['SHIP_ID'] == ship_id, 'PRED'] = pred
    else:
        # 새로운 행 추가
        new_row = {'SHIP_ID': ship_id, 'PRED': pred}
        data = data.append(new_row, ignore_index=True)
        
    # 현재 날짜 가져오기
    current_date_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    # 문자열을 다시 datetime 객체로 변환
    current_date = datetime.strptime(current_date_str, "%Y-%m-%d %H:%M:%S")
    data['DT_UPDATE'] = current_date
    
    # 중복을 피하기 위해 기존 데이터 삭제
    delete_from_database('tc_ai_ecu_cleaning_cycle_model')
    
    # 수정된 데이터를 데이터베이스에 다시 저장
    load_database(data, 'tc_ai_ecu_cleaning_cycle_model')

In [22]:
def caculate_prediction_value(ship_name, ship_id):
    time_data = ConfiguringTimeData(ship_name)
    data = time_data.group_3h_time_series()
    
    time_model = TimeSeriesModel(data)

    predict = time_model.predict()
    
    create_dataset(ship_id,predict)
    
ship_name = 'BULK LAMBERT'
ship_id = 'T20191002002'

caculate_prediction_value(ship_name, ship_id)

모델이 성공적으로 'C:\Users\pc021\Desktop\dx_project\techross\health_learning_data\time_model\src\my_package\model\time_model_250_3h_10.pkl'에서 로드되었습니다.
모델이 성공적으로 'C:\Users\pc021\Desktop\dx_project\techross\health_learning_data\time_model\src\my_package\scaler'에서 로드되었습니다.
[[[1.        ]]

 [[0.99838397]]

 [[0.98970318]]

 [[0.99761171]]

 [[0.96641377]]

 [[0.85841872]]

 [[0.98222368]]

 [[0.26777453]]

 [[0.        ]]

 [[1.        ]]]
1/1 [==============================] - 0s 118ms/step


C:\Users\pc021\.pyenv\pyenv-win\versions\3.8.10\lib\site-packages\sklearn\base.py:299: UserWarning: Trying to unpickle estimator MinMaxScaler from version 1.3.2 when using version 1.2.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\pc021\AppData\Local\Temp\ipykernel_16496\2339041978.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(new_row, ignore_index=True)


Successfully deleted all records from tc_ai_ecu_cleaning_cycle_model. Deleted rows: 4
DataFrame has been successfully loaded into tc_ai_ecu_cleaning_cycle_model table in ecs_dat1 database.
